In [1]:
%pip install import-ipynb
%pip install openpyxl
%pip install xlrd

import import_ipynb
import load_data
import clean_column_names
# import write_file_as_output
# import normalize_strings
# import fill_missing_values
# import text_to_numeric
# import drop_columns
import pandas as pd
import os
import re


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# LOAD DATA NEW

# Load Data
def load_file(file_path):
    print(file_path)

    # converts filepath extension to lowercase
    ext = os.path.splitext(file_path)[-1].lower()

    # Handle .csv files
    """pd.read_csv expects a file path as its argument - above line of code will not work
    because it is being passed a DataFrame/ Series object created by load file method and saved
    to variable named 'file'
    """
    if ext in ['.csv']:
        return pd.read_csv(file_path)

    # Handle excel spreadsheet files
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path, engine="openpyxl")|pd.read_excel(file_path, engine="xlrd")
    
    # All other files
    else:
        raise ValueError("Unsupported file type")


    

In [3]:
# WRITE CLEANSED FILE AS OUTPUT
def output_file(df):
    # MAKE SURE TO LIST FULL PATH TO OUTPUT FILE OTHERWISE FILL WILL BE WRITTEIN IN DIRECTORY IN WHICH THIS FILE RESIDES!
    df.to_csv(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_CLEANSED.csv')
    print(f"Cleaned data saved!")

In [7]:

# Create DataFrame from file at specified filepath: NOTE- This calls loaf_file() function
def create_dataframe(file_path):
    # df = load_file(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop - ORIGINAL_xl.csv')
    df = load_file(file_path)

    #Create FILE TO BE CLEANSED via deep copy of original df to new variable 
    deep_copied_df = df.copy()
    print(f"FILE READY FOR CLEANSING\n")
    
    print(deep_copied_df.info(),"\n")
    print(deep_copied_df.shape, "\n")
    return deep_copied_df

# CREATE DataFrame - ORIGINAL - DO NOT MODIFY
    # df = pd.read_csv(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_COPY.xlsx')
    # df = pd.read_excel(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_COPY.xlsx')



In [8]:
cleansed_file = create_dataframe(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop - ORIGINAL_xl.csv')

C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop - ORIGINAL_xl.csv
FILE READY FOR CLEANSING

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STOP ID          1674 non-null   float64
 1   NAME             1674 non-null   object 
 2   BENCH            444 non-null    object 
 3   SHELTER          1674 non-null   object 
 4   SHELTER TYPE     176 non-null    object 
 5   RIDERSHIP ON     1586 non-null   float64
 6   RIDERSHIP OFF    1586 non-null   float64
 7   RIDERSHIP TOTAL  1586 non-null   float64
 8   STOP DIRECTION   1671 non-null   object 
 9   ROUTE NO.        1674 non-null   float64
 10  ROUTE NO. 1      285 non-null    float64
 11  ROUTE NO. 2      54 non-null     float64
 12  ROUTE NO. 3      16 non-null     float64
 13  ROUTE NO. 4      12 non-null     float64
 14  ROUTE NO. 5      11 non-nu

In [5]:


# Copy of original data file
# file_toBe_cleansed = df.copy()

# cleansed_file = load_file(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_COPY.xlsx')
# print("FILE READY FOR CLEANSING")
# print("FILE LOADED: \n\n ", file_toBe_cleansed, "\n")

#cleansed_file = clean_column_names.clean_column_names(file_toBe_cleansed)
# print("CLEANSED FILE: \n" ,cleansed_file, "\n")

#cleansed_file = normalize_strings.normalize_strings(file_toBe_cleansed)
# print("CLEANSED FILE w/ strings normalized: \n" ,cleansed_file ,"\n")

# #cleansed_file = fill_missing_values.fill_missing(file_toBe_cleansed)
# print("CLEANSED FILE w/ missing values filled: \n" ,cleansed_file ,"\n")

# #cleansed_file = text_to_numeric.text_to_numeric(file_toBe_cleansed)
# print("CLEANSED FILE w/ numbers converted to numeric values: \n" ,cleansed_file ,"\n")

# cleansed_file = drop_columns.drop_columns(file_toBe_cleansed)
# print("CLEANSED FILE: COLUMNS DROPPED: \n" ,cleansed_file ,"\n")

# write_file_as_output.output_file(cleansed_file)




# 8 Most Common Data Cleansing Techniques:
## 1. Dropping Columns:
cols_to_drop = ["column_names_to_be_dropped"] > df_copy.drop(to_drop, axis = 1) NOTE: axis = 0 (rows) axis = 1 (columns)

In [6]:
# Drop irrelevant/ unnecessary columns
cols_to_drop = ["ROUTE NO. 1",
               'ROUTE NO. 2','ROUTE NO. 3',
               'ROUTE NO. 4','ROUTE NO. 5','ROUTE NO. 6',
                'ROUTE NO. 7','ROUTE NO. 8','ROUTE NO. 9','ROUTE NO. 10',
               "GEOMETRY",]

cleansed_file = cleansed_file.drop(cols_to_drop, axis = 1)
output_file(cleansed_file)

Cleaned data saved!


# 2. Removing Duplicates
## Info - https://www.w3schools.com/python/pandas/ref_df_duplicated.asp

Syntax: <br>
get duplicates returned in DataFrame Format: df_copy[df_copy.duplicated()] <br>
get duplicates returned in Text Format: df_copy.duplicated()

In [7]:
# Sum total number of duplicate rows in DataFrame BEFORE dropping dupes
# Write this to report for client later
cleansed_file.duplicated().sum()

np.int64(24)

In [8]:
# Drop duplicates
cleansed_file.drop_duplicates(inplace=True)

In [9]:
# Drop duplicate rows from DataFrame + Sum total number of rows dropped
cleansed_file = cleansed_file.drop_duplicates()
output_file(cleansed_file)

Cleaned data saved!


In [10]:
cleansed_file

,STOP ID,NAME,BENCH,SHELTER,SHELTER TYPE,RIDERSHIP ON,RIDERSHIP OFF,RIDERSHIP TOTAL,STOP DIRECTION,ROUTE NO.
0,1193.0,Mills @ Scenic Hwy,NB,NO,NaN,511.0,463.0,974.0,S,11.0
1,2824.0,Cedarcrest Ave @ Seracedar St,NaN,NO,NaN,NaN,NaN,NaN,S,58.0
2,1191.0,Woodpecker @ Merganzer,NaN,NO,NaN,479.0,153.0,632.0,S,11.0
3,1177.0,Blount @ Tallow,NaN,NO,NaN,319.0,460.0,779.0,W,11.0
4,1593.0,Acadian @ Oswego,NaN,NO,NaN,166.0,440.0,606.0,N,20.0
...,...,...,...,...,...,...,...,...,...,...
1694,1105.0,Scenic @ Scotlandville Library,NaN,YES,REFURBISHED,1814.0,790.0,2604.0,S,10.0
1695,1996.0,Florida @ Eugene,NaN,NO,NaN,824.0,3722.0,4546.0,W,44.0
1696,1514.0,East Airport @ Goodwood,NB,NO,NaN,123.0,117.0,240.0,W,18.0
1697,1386.0,Thomas H Delpit @ Taft,NaN,NO,NaN,163.0,41.0,204.0,S,14.0


# 3. Removing/ Dropping Irrelevant/ Unnecessary Rows
Syntax Ex: <br>
Check for Jobs located in State of Ohio in DataFrame: <br>
df_copy['Location'].str.contains("Oh", case=False) <br> <br>
df_copy['solumn_Name'].str.contains("specified_String", case=False) <br>


# 4. Removing Nulls and Blanks

## https://www.w3schools.com/python/pandas/ref_df_dropna.asp <br>
NOTE: df_dropna() method offers more flexibility in what is dropped. <br>
## https://www.w3schools.com/python/pandas/ref_df_drop.asp <br>


## Get count of na (Not a Number values) - df_copy.isna().sum <br>

## Get count of null (null values) - df_copy.isnull().sum<br>
## Get count of null / na values fpr specific column - df["column name"].isna() / df['column name'].isnull()<br>
## For multiple columns pass a list of column names in - df.[["list_of_column_names"]].isna()


In [11]:
# Example: Drop rows where all values are null/blank
cleansed_file.dropna(axis=0, how='all', inplace=True)

In [12]:
output_file(cleansed_file)

Cleaned data saved!


# 4B - Filling Nulls / Blanks
## https://www.w3schools.com/python/pandas/ref_df_fillna.asp

In [13]:
# Cast value as appropriate data type before assignment
# cleansed_file.loc[0, ['RIDERSHIP ON', 'RIDERSHIP OFF', 'RIDERSHIP TOTAL']] = float(0)

for column in cleansed_file:
    if cleansed_file[column].dtype == 'object':
        cleansed_file[column].fillna("N/A", inplace=True)
    cleansed_file[column].fillna(0, inplace=True)

C:\Users\Cessn\AppData\Local\Temp\ipykernel_16712\130094186.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleansed_file[column].fillna(0, inplace=True)
C:\Users\Cessn\AppData\Local\Temp\ipykernel_16712\130094186.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [14]:
output_file(cleansed_file)

Cleaned data saved!


# 5 - Standardizing Values (6:26)

# SYNTAX Examples: <br>
# NOTE use df.loc as shown below to get specific column info: <br>
df_copy.loc[row_indexer, ["column_name"]] <br>
df_copy.loc[0:, ["column_name"]]

In [15]:
# NOTE use df.loc as shown below to get specific column info
# df_copy.loc[0:, ["Job Title"]]

In [16]:
# Return all elements in Job Title column where Job Title string contains a specific word
# df_copy[df_copy["Job Title"].str.contains('scientist', case=False)]


In [17]:
# Standardize all strings containing word scientist to read "Data Scientist"
# df_copy.loc[df_copy["Job Title"].str.contains("scientist", case=False), "Job Title"] = "Data Scientist"

In [18]:
# Check to ensure fields in Job Title Column have been updated
# df_copy.loc[:, "Job Title"]

In [19]:
# Capture /show all strings containing the letter k or K in column Salary Estimate 
# df_copy.loc[df_copy["Salary Estimate"].str.contains("k", case=False), "Salary Estimate"]

In [20]:
# Replace letter "k" Salary Estimate column with 000 
# Final value: 181K - 182K > 181000 - 182000

# Use dictionay object as per below syntax examples:

# replace_dict ={"\$": "", 
#                "K": "000",
#               }
# df_copy["Salary Estimate"] = df_copy['Salary Estimate'].replace(replace_dict, regex=True)

In [21]:
# df_copy["Salary Estimate"]

# 6 - Splitting Columns 8:16
## Documentation: https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html <br>
## Youtube: https://youtu.be/FbFQH_RNMu0?t=496

## Example Code: https://docs.google.com/document/d/15Ajf-MKtOWItDITu-kukbEeshtCyMC86htXgTiMZJOA/edit?tab=t.0

## SYNTAX EXAMPLES:


# 7 - Convert Datatypes <br>
## Youtube: https://youtu.be/FbFQH_RNMu0?t=613 <br>
## Code Examples: https://docs.google.com/document/d/1HiZYEBH_5RaNjvvVtOodOFB0z-ji_4kx-kF9pv1IE-U/edit?tab=t.0 <br>

## Convert datatype: multiple columns to datatype int: <br>
convert_dtypes ={'colName_1': 'int',
                'colName_2': 'int',} <br>
df = df.astype(convert_dtypes) <br>

# CONVERT datatype of single column <br>
df['colName_1'] = df['colName_1'].astype("specify_datatype")

In [24]:
# CONVERT RIDERSHIP columns to data type INT from datatype FLOAT:
convert_dtypes ={'RIDERSHIP ON': 'int',
                'RIDERSHIP OFF': 'int',
                'RIDERSHIP TOTAL': 'int',}
cleansed_file = cleansed_file.astype(convert_dtypes)

In [25]:
output_file(cleansed_file)

Cleaned data saved!


# 8 - Stripping Strings <br>
## Youtube - https://youtu.be/FbFQH_RNMu0?t=648 <br>

## SYNTAX EXAMPLES
## Cleaning special / non-word characters from end of string: <br>
df = dfloc[:, 'column_Name'].str[:-4] - NOTE this cleans up to the last 4 characters of the string